# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
imported_csv = "output_data/cities.csv"
new_weather_df = pd.read_csv(imported_csv)
new_weather_df.head()


,Unnamed: 0,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed (mph)
0,0,Hilo,19.73,-155.09,66.90,73,90,4.70
1,1,Carnarvon,-24.87,113.63,59.00,38,0,8.05
2,2,Cape Town,-33.93,18.42,54.36,66,20,6.93
3,3,Chitradurga,14.23,76.40,76.68,74,96,15.79
4,4,Manbij,36.53,37.95,91.40,41,1,14.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = new_weather_df[["Lat", "Lon"]].astype(float)
humidity = new_weather_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
perfect_weather_df = new_weather_df.loc[(new_weather_df["Temp"]<= 80)].copy()
#[(new_weather_df["Temp"]> 65)][(new_weather_df["Humidity"]< 60)]

hotel_df = perfect_weather_df

hotel_df["Hotel Name"] = ""
hotel_df.head()

,Unnamed: 0,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed (mph),Hotel Name
0,0,Hilo,19.73,-155.09,66.90,73,90,4.70,
1,1,Carnarvon,-24.87,113.63,59.00,38,0,8.05,
2,2,Cape Town,-33.93,18.42,54.36,66,20,6.93,
3,3,Chitradurga,14.23,76.40,76.68,74,96,15.79,
5,5,Mar del Plata,-38.00,-57.56,50.85,57,39,23.04,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#params
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Identify Hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lon = row["Lon"]
    target_city = row["City"]

    # iterate locations
    params["location"] = f"{lat},{lon}"

    # Set Base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()

    # Fill countries without available hotel with "Not Available".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]

    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "Not Available"

hotel_df

,Unnamed: 0,City,Lat,Lon,Temp,Humidity,Cloudiness,Wind Speed (mph),Hotel Name
0,0,Hilo,19.73,-155.09,66.90,73,90,4.70,Hilo Hawaiian Hotel
1,1,Carnarvon,-24.87,113.63,59.00,38,0,8.05,Hospitality Carnarvon
2,2,Cape Town,-33.93,18.42,54.36,66,20,6.93,Southern Sun Waterfront Cape Town
3,3,Chitradurga,14.23,76.40,76.68,74,96,15.79,V. G. S. Comforts
5,5,Mar del Plata,-38.00,-57.56,50.85,57,39,23.04,Gran Hotel Mar del Plata habitaciones
...,...,...,...,...,...,...,...,...,...
564,564,Paraná,-31.73,-60.52,57.49,36,6,6.15,Howard Johnson Plaza Hotel Mayorazgo
566,566,Ekibastuz,51.67,75.37,70.65,52,10,5.08,Not Available
567,567,Bafang,5.16,10.18,65.80,98,99,3.36,Motel Mary-Alex
572,572,Sioux Lookout,50.10,-91.92,62.60,77,75,5.82,Knobby's Fly-In Camps


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>

</dl>
"""
#<dt>Country</dt><dd>{Country}</dd>
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in perfect_weather_df.iterrows()]
locations = perfect_weather_df[["Lat", "Lon"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))